### set parameters

In [1]:
confidence_thresh = 0.5
conf_name = '0_5'
output_name = 'yolo_m_weighted_aug'
columns_suffix = '_m_weighted_aug'
output_folder = r"C:\Users\janni\Dokumente\Masterarbeit\model_results\yolo_m_weighted_sampler_with_augmentations\val_data_metrics_conf_0_5"

yolo_model = r"C:\Users\janni\Dokumente\Masterarbeit\models\tree_species_model\yolo_m_weighted_sampler_with_augmentations\runs\segment\train\weights\best_yolo_m_weighted_sampler_augment.pt"
img_dataset = r"C:\Users\janni\Dokumente\Masterarbeit\images\combined_annotated_images\tree_species_image_data\data.yaml"

# create folder if it does not exist
from pathlib import Path
Path(output_folder).mkdir(parents=True, exist_ok=True)

### load model and do validation

In [2]:
from ultralytics import YOLO

model = YOLO(yolo_model)

metrics = model.val(data=img_dataset,
                    conf=confidence_thresh, 
                    project = output_folder,
                    name = f'val_{output_name}_conf_{conf_name}')

Ultralytics YOLOv8.0.199  Python-3.11.5 torch-2.1.0+cpu CPU (AMD Ryzen 7 5700U with Radeon Graphics)
YOLOv8m-seg summary (fused): 245 layers, 27235701 parameters, 0 gradients, 110.0 GFLOPs
val: Scanning C:\Users\janni\Dokumente\Masterarbeit\images\combined_annotated_images\tree_species_image_data\valid\labels.cache... 1569 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1569/1569 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Mask(P          R      mAP50  mAP50-95): 100%|██████████| 99/99 [09:06<00:00,  5.52s/it]
                   all       1569       1607      0.922      0.917      0.945      0.929      0.921      0.916      0.945      0.936
            abies_alba       1569         27      0.909      0.741      0.851      0.828      0.909      0.741      0.851      0.849
        acer_campestre       1569         19          1      0.947      0.974      0.973          1      0.947      0.974      0.973
      acer_plata

### extract necessary validation metrics for all classes and each class seperatly

In [3]:
# copy the confusion matrix to desired output folder and rename
import os
import shutil

# get conf and iou of confusion matrix
confidence_mat = metrics.confusion_matrix.conf
iou_mat = metrics.confusion_matrix.iou_thres

# get files of confusion matrix
confusion_mat = os.path.join(output_folder, f'val_{output_name}_conf_{conf_name}', 'confusion_matrix.png')
confusion_mat_normal = os.path.join(output_folder, f'val_{output_name}_conf_{conf_name}', 'confusion_matrix_normalized.png')

# Specify destination (including the new file name)
dest_conf_mat = os.path.join(output_folder, f"confusion_matrix_conf_{confidence_mat}_iou_{iou_mat}.png")
dest_conf_mat_normal = os.path.join(output_folder, f"confusion_matrix_normalized_conf_{confidence_mat}_iou_{iou_mat}.png")

# Copy the file
shutil.copy2(confusion_mat, dest_conf_mat)
shutil.copy2(confusion_mat_normal, dest_conf_mat_normal)

'C:\\Users\\janni\\Dokumente\\Masterarbeit\\model_results\\yolo_m_weighted_sampler_with_augmentations\\val_data_metrics_conf_0_5\\confusion_matrix_normalized_conf_0.5_iou_0.45.png'

##### metrics for per class

In [29]:
# get class indexes
class_index = metrics.ap_class_index

# box precisions for all classes
p_box = metrics.box.p

# box recall for all classes
r_box = metrics.box.r

# box f1 for all classes
f1_box = metrics.box.f1

# box average precision for all classes, iou thresholds between 0.5 and 0.95: ap50-95
ap_box = metrics.box.ap

# box average precision for all classes, iou threshold 0.5: ap50
ap50_box = metrics.box.ap50

# mask precisions for all classes
p_mask = metrics.seg.p

# mask recall for all classes
r_mask = metrics.seg.r

# mask f1 for all classes
f1_mask = metrics.seg.f1

# mask average precision for all classes, iou thresholds between 0.5 and 0.95: ap50-95
ap_mask = metrics.seg.ap

# mask average precision for all classes, iou threshold 0.5: ap50
ap50_mask = metrics.seg.ap50

##### mean metrics for all classes

In [30]:
# box mean precision
mp_box = metrics.box.mp

# box mean recall 
mr_box = metrics.box.mr

# box mean average precision for all classes, iou thresholds between 0.5 and 0.95: ap50-95
map_box = metrics.box.map

# box mean average precision for all classes, iou threshold 0.5: ap50
map50_box = metrics.box.map50


# mask mean precision
mp_mask = metrics.seg.mp

# mask mean recall 
mr_mask = metrics.seg.mr

# mask mean average precision for all classes, iou thresholds between 0.5 and 0.95: ap50-95
map_mask = metrics.seg.map

# mask mean average precision for all classes, iou threshold 0.5: ap50
map50_mask = metrics.seg.map50

#### combine to one dataframe

In [31]:
import os
import pandas as pd
import numpy as np

dataset = pd.DataFrame({'class_index': class_index,
                        f'precision_box{columns_suffix}': p_box,
                        f'recall_box{columns_suffix}': r_box,
                        f'map_50_box{columns_suffix}': ap50_box,
                        f'map_50_95_box{columns_suffix}': ap_box,
                        f'f1_box{columns_suffix}': f1_box,
                        f'precision_mask{columns_suffix}': p_mask,
                        f'recall_mask{columns_suffix}': r_mask,
                        f'map_50_mask{columns_suffix}': ap50_mask,
                        f'map_50_95_mask{columns_suffix}': ap_mask,
                        f'f1_mask{columns_suffix}': f1_mask})


dataset_mean = pd.DataFrame({'class_index': 'all',
                             f'precision_box{columns_suffix}': mp_box,
                             f'recall_box{columns_suffix}': mr_box,
                             f'map_50_box{columns_suffix}': map50_box,
                             f'map_50_95_box{columns_suffix}': map_box,
                             f'f1_box{columns_suffix}': np.mean(f1_box),
                             f'precision_mask{columns_suffix}': mp_mask,
                             f'recall_mask{columns_suffix}': mr_mask,
                             f'map_50_mask{columns_suffix}': map50_mask,
                             f'map_50_95_mask{columns_suffix}': map_mask,
                             f'f1_mask{columns_suffix}': np.mean(f1_mask)}, index=[0])

final_dataset = pd.concat([dataset_mean, dataset], ignore_index=True)

# species data frame
species_classes = pd.DataFrame(list(model.names.items()), columns=['class_index', 'species'])

species_all = pd.DataFrame({'class_index': 'all', 'species': 'all'}, index=[0])

species_df = pd.concat([ species_all, species_classes], ignore_index=True)

# combine final df and species df
out_df = species_df.merge(final_dataset, on="class_index")

filename = f'metrics_{output_name}_conf_{conf_name}.txt'

output_file_path = os.path.join(output_folder, filename)

out_df.to_csv(output_file_path, sep='\t', index=False)